# Dataloader

Just a notebook showing that the dataloader can be imported and it loads the data

In [ ]:
# Add the src folder to the path
import sys
sys.path.insert(0, '../src/')

from data.dataloader import MidiDataset
from data.bar_transform import BarTransform

## Initialize transform as a function to put into the dataloader

In [ ]:
transform = BarTransform(bars=2, note_count=60)
transform

In [ ]:
midi_dataset = MidiDataset(csv_file='./concat.csv', transform = transform)
midi_dataset.__len__()

## Get the first item of the dataset

In [ ]:
first = midi_dataset.__getitem__(0)
first

## Iterating the dataset

In [ ]:
for i in range(len(midi_dataset)):
    sample = midi_dataset[i]
    print("{}, {} samples".format(i, len(sample['piano_rolls'])))

    if i == 10:
        break

In [ ]:
midi_dataset.get_mem_usage()